# M2E03: Interfacing Python with SQLite and MySQL

# Data Science Master 2021/22

# Prof. Dr. Gottfried Vossen

In this notebook we discuss how to interface Python and databases, by way of two examples, SQLite and MySQL. We focus on these two systems for several reasons: The principle of interfacing Python and a database is similar for all systems. SQLite is the most widely used database system worldwide (and available in every smartphone); MySQL is a widely used free relational system. A more comprehensive introduction to working with databases in Python can be found at https://realpython.com/tutorials/databases/.

Make sure that both SQLite and MySQL are properly installed on your machine before you continue; instructions to do so are referenced in this notebook.

## Contents

1. <a href='#sqlite'>Interfacing Python and SQLite</a> 

2. <a href='#mysql'>Interfacing Python and MySQL</a>

<a name='sqlite'></a>
## 1. Interfacing Python and SQLite <a class="anchor" id="sqlite"/>

<div style='align: left; text-align:center;'>
    <img src='img/SQLite.png' alt='SQLite' style="height:200px;"/>
    <span style='display:block;'>Image Source: <a href="https://en.wikipedia.org/wiki/SQLite" target="_blank">Wikipedia</a></span>
    <br/>
</div>

A common and popular data access pattern with Pandas is to query data from a database directly into a Pandas DataFrame.  Once the data is in the DataFrame, a user can further analyze the data. We will look at this feature with SQLite as well as MySQL as sample databases (see https://www.sqlitetutorial.net/sqlite-python/ as wel as https://pypi.org/project/MySQL-python/ for references); for other databases such as PostgreSQL, DB2, or Oracle you need to refer to https://www.sqlalchemy.org/ for the proper packages to install.

In general, there are three ways you can install packages:
* You can use Anaconda Navigator
* You can use "conda install <package-name>" from the terminal or anaconda prompt
* You can use "pip install <package-name>" from the terminal or anaconda prompt

In [ ]:
# First we need to import the proper packages:

import sqlite3
import pandas as pd


### Defining the SQLite connector class

Here, we define the `SQLiteDBConnector` class that encapsulates everything we need to work with SQLite in Python. This class helps us to connect to a SQLite database stored in `db_filepath` (e.g., `~\database\mydatabase.db`) or creates one if it doesn't exist.

`SQLiteDBConnector` also helps us to retrieve metadata from the database such as table names by using the method `.get_table_names()`.

More information about how to use SQLite with Python can be found at the [SQLite Tutorial](https://www.sqlitetutorial.net/sqlite-python/).


In [ ]:
class SQLiteDBConnector(object):
    def __init__(self, db_filepath):
        self.db_filepath = db_filepath
        self.open_connection()
        
    def open_connection(self):
        self.connection = self.create_connection(self.db_filepath)
        
    def create_connection(self, db_filepath):
        """ Create a database connection to the SQLite database
            specified by the db_file
        :param db_file: database file
        :return: Connection object
        """
        connection = sqlite3.connect(db_filepath)
        
        return connection
    
    def close_connection(self):
        self.connection.close()
    
    def execute_query_to_pandas(self, query):
        """ Execute a SQL query and accomodate the result in a Pandas Dataframe
        :param query: SQL query
        :return: Pandas DataFrame object
        """
        result = pd.DataFrame()
        try:
            with self.connection: 
                result = pd.read_sql_query(query, self.connection)
        except pd.io.sql.DatabaseError as e:
            print('------------- Query Error ----------------')
            print(e)
            print('------------------------------------------')
        return result

    def execute_command(self, sql_command):
        """ Execute a SQL query and return its result
        :param sql_command: SQL query
        :return: Query result
        """
        result = None
        
        cs = self.connection.cursor()
        cs.execute(sql_command)
        result = cs.fetchall()
        cs.close()

        return result

    def get_table_names(self):
        """ Get names of all tables in the connected SQLite database
        :return: List of table names
        """
        table_names = []
        
        with self.connection:
            cs = self.connection.cursor()
            cs.execute("SELECT name FROM sqlite_master WHERE type='table';")
            table_names = [t[0] for t in cs.fetchall() if not t[0] in ['sqlite_master', 'global_inverted_index', 'sqlite_sequence']]
            cs.close()
        
        return table_names

### Creating a SQLite database file

In [ ]:
dbconnector = SQLiteDBConnector('Data\\cats.db')

### Defining a table

In [ ]:
dbconnector.execute_command('DROP TABLE IF EXISTS cats;')
dbconnector.execute_command('CREATE TABLE cats(owner VARCHAR(10), name VARCHAR(10), type SMALLINT);')

### Inserting data

In [ ]:
dbconnector.execute_command("INSERT INTO cats VALUES('juan', 'darcy', 1);")
dbconnector.execute_command("INSERT INTO cats VALUES('bob', 'lucky', 2);")
dbconnector.execute_command("INSERT INTO cats VALUES('bob', 'oreo', 1);")

### A simple SELECT statement

In [ ]:
dbconnector.execute_command("SELECT * FROM cats;")

### The query result as a Pandas DataFrame

In [ ]:
dbconnector.execute_query_to_pandas("SELECT * FROM cats;")

In [ ]:
# A simple projection

dbconnector.execute_query_to_pandas("SELECT owner, name FROM cats;")

In [ ]:
# Combining selection and projection

dbconnector.execute_query_to_pandas("SELECT owner, name FROM cats where type = 1;")

In [ ]:
# Counting

dbconnector.execute_query_to_pandas("SELECT COUNT(*) FROM cats;")

In [ ]:
# Group-by

dbconnector.execute_command("SELECT owner, COUNT(*) FROM cats GROUP BY owner;")

In [ ]:
# Ordering

dbconnector.execute_command("SELECT owner, COUNT(*) FROM cats GROUP BY owner ORDER BY COUNT(*) ASC;")

In [ ]:
# Getting table names

dbconnector.execute_command("SELECT * FROM sqlite_master WHERE type='table';")

# sqlite_master has the following attributes: type (object type), name (object name), 
# tbl_name (table name), rootpage, and sql (DDL statement)
# additional information is available at https://sqlite.org/fileformat.html


In [ ]:
# Use of the get_table_names() method to get a list of table names as return value

dbconnector.get_table_names()

### Closing the database connection

In [ ]:
dbconnector.connection.close()

# From this point on, we can no longer execute database commands.

In [ ]:
# But we can reopen the connection ...

dbconnector.open_connection()

In [ ]:
# ... and go again

dbconnector.execute_command("SELECT * FROM cats;")


In [ ]:
# Yet when we don't need it anymore, we close it
dbconnector.connection.close()


### Creating an SQLite database from a .csv file

### Load csv data into a DataFrame

In [ ]:
dataframe = pd.read_csv('Data\\auto-mpg-tabs.csv', sep='\t', index_col=0)
dataframe.head()

### Open connection/create sqlite database

In [ ]:
dbconnector = SQLiteDBConnector('Data\\auto.db')

### Export DataFrame to SQLite

In [ ]:
dbconnector.execute_command('DROP TABLE IF EXISTS auto;')
dataframe.to_sql(name='auto', con=dbconnector.connection)

### Execute a query

In [ ]:
dbconnector.execute_command("SELECT * FROM auto;")

### Import query result to a Pandas DataFrame

In [ ]:
dbconnector.execute_query_to_pandas("SELECT * FROM auto;")

In [ ]:
# Finally, close the connection again
dbconnector.connection.close()

### Exercise

In the cells below, first load the SQLite `cats.db` into a pandas DataFrame and the use the exploratory methods `describe()` and `info()`.


dbconnector = SQLiteDBConnector('Data\\cats.db')

catsframe = dbconnector.execute_query_to_pandas('SELECT * FROM cats')

catsframe

catsframe

For more on SQLite, take a look at https://towardsdatascience.com/python-has-a-built-in-database-heres-how-to-use-it-47826c10648a.

## 2. Interfacing Python and MySQL <a class="anchor" id="mysql"/>

<div style='align: left; text-align:center;'>
    <img src='img/mysql.png' alt='MySQL' style="height:150px;"/>
    <span style='display:block;'>Image Source: <a href="https://en.wikipedia.org/wiki/MySQL" target="_blank">Wikipedia</a>.</span>
    <br/>
</div>

Working with SQLite is fun and really helpful when your application runs on a limited environment (e.g., an embedded system, a smartphone app, a mobile game). For more complex applications, however, you will need a more powerful database management system (DBMS). One of the most popular alternative, used by many companies including Facebook, Google, and Adobe, is  MySQL. MySQL is a central component of [LAMP](https://en.wikipedia.org/wiki/LAMP_%28software_bundle%29) (i.e., **L**inux OS, **A**pache HTTP Server, **M**ySQL relational DBMS, and **P**HP programming language), which is a software stack model used in of a plethora of Web applications.

### Download and Installation

In this course module, we use [MySQL Community Server](https://dev.mysql.com/downloads/mysql/), an open source (GPL) version. To download and install MySQL, follow the installation instructions from the [MySQL's official Website](https://dev.mysql.com/doc/mysql-getting-started/en/#mysql-getting-started-installing).

You can also follow the video tutorial below:

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('u96rVINbAUI', width=600, height=350)

To use MySQL via Python, we use a MySQL driver that can be downloaded from [MySQL's official Website](https://dev.mysql.com/downloads/connector/python/). Alternatively, you can use the following **pip** command line:

```bash
pip install mysql-connector-python
```

or 

```bash
python3 -m pip install mysql-connector-python
```

### Getting started

With MySQL and the Python Connector installed, we can start a connection to the database. We start by importing `mysql.connector` and calling the `connect()` method. This method requires 3 parameters `host` (the db server address, for a local installation, `localhost`), `user` (the username needeed to connect with the db, we use `root` here), and `passwd` (the password associated with the current user, you defined the password for user `root` during the installation). 

In [ ]:
import mysql.connector as mysql

In [ ]:
# 'connect()' method requires 3 parameters 'host', 'user', 'passwd'
dbconnection = mysql.connect(
    host = "localhost",
    user = "root",
    passwd = "vossensql"
)

Let's print `dbconnection` to make it sure everything works fine. The print output of should produce something like `<mysql.connector.connection.MySQLConnection object at 0x000001EB5DE0CB48>`. If that's the case, we have successfully connected to MySQL database. Otherwise, check username and password information.

In [ ]:
print(dbconnection)

Now, we create a cursor that is used to execute SQL statements. Let's list all existing databases using the `execute()` method. `execute()` takes in a string with a SQL statement to be executed. 

In [ ]:
# Creating a cursor to execute SQL statements
cursor = dbconnection.cursor()

# 'execute()' method is used to compile a SQL statement
cursor.execute("SHOW DATABASES")

The SQL statement `SHOW DATABASES` produces a result that can be inspected by using the `fetchall()` method. `fetchall()` fetches all the rows from the last executed statement. Once we call this method, the result if depleted and we need to call the `execute()` method again.

In [ ]:
## 'fetchall()' method fetches all the rows from the last executed statement
databases = cursor.fetchall() ## it returns a list of all databases present

## printing the list of databases
print(databases)

Let's create now a new database called **dbpy** by calling the `execute()` method. Note that a if a database with this name already exists, you will get an error message. To avoid that, we first check if a database with the same name exists and delete it using the SQL command `DROP DATABASE`.

In [ ]:
# Deleting the `dbpy` database if it already exists.
cursor.execute("DROP DATABASE IF EXISTS dbpy")

# Creating a new database.
cursor.execute("CREATE DATABASE dbpy")

To work with our new **dbpy** database, we need to create a new connection specifying that database.

In [ ]:
dbpy = mysql.connect(
    host = "localhost",
    user = "root",
    passwd = "vossensql",
    database = "dbpy"
)

With that new connection, let's create a cursor to start executing SQL statements.

In [ ]:
cursor = dbpy.cursor()

Following the step we performed before with SQLite, we create a new table called `cats`.

In [ ]:
# Creating a new table called cats.
cursor.execute("CREATE TABLE cats(owner VARCHAR(10), name VARCHAR(10), type SMALLINT)")

Let's insert some data into the table.

In [ ]:
cursor.execute("INSERT INTO cats VALUES('juan', 'darcy', 1)")
cursor.execute("INSERT INTO cats VALUES('bob', 'lucky', 2)")
cursor.execute("INSERT INTO cats VALUES('bob', 'oreo', 1)")

Now we can start querying the database.

In [ ]:
cursor.execute("SELECT owner, name FROM cats")
catsinfo = cursor.fetchall() # returns tuples from the cats table
for c in catsinfo:
    print(c)

In [ ]:
cursor.execute("SELECT owner, COUNT(*) FROM cats GROUP BY owner ORDER BY COUNT(*) ASC")
catsinfo = cursor.fetchall() # returns the result of the execute method
for c in catsinfo:
    print(c)

### Using Pandas with MySQL 

We can query our MySQL database directly from Pandas. To do that, we use the `read_sql()` method that takes in 2 parameters, namely, the SQL query we want to execute and the database connection. Note that instead of a list of results, `read_sql` returns a Pandas data frame.

In [ ]:
import pandas as pd

pd.read_sql('SELECT owner, COUNT(*) FROM cats GROUP BY owner ORDER BY COUNT(*) ASC', dbpy)

### Working with a sample database

Here we use the **MySQL Employees sample database**. This database consists of 4 million records in total. You can check how its schema looks like [here](https://dev.mysql.com/doc/employee/en/sakila-structure.html).

First, you have to download the GitHub repository [test_db](https://github.com/datacharmer/test_db) as a .zip file. Extract the content of the .zip file in a directory called **testdb**.

Second, open a command line tool (Windows) or terminal/bash (macOS or Ubuntu) and type in the command below. Enter the password of your MySQL user account (configured during MySQL installation). 

```bash
mysql -uroot -p
```

**Note**: You might need to add the MySQL directory to the PATH variable (Environment Variables). Alternatively, you can use your command line tool or terminal/bash to navigate to the MySQL directory where the mysql application is installed. On Windows, this directory is commonly located at `C:\Program Files\MySQL\MySQL Server 8.0\bin`. 
In case you have problems with the installation, follow the instructions in [this video](https://www.youtube.com/watch?v=eP_W-nbpYIo).

If everything worked correctly, you will see the mysql command line tool as depicted below, which means it is ready to be used:

```bash
mysql > 
```

Finally, you need to run the `employees.sql` file stored in the **testdb** directory. To do that, enter the command below. Note that you need to specify the correct path to the **testdb** directory. As an example, this directory might located at `C:\test_db\employees.sql`.

```bash
source C:\testdb\employees.sql
```

When the command finishes, you can check if the database was correctly installed by typing in the command below.

```bash
mysql > use employees;
```

If you got the message "Database changed" in response to the command, then the installation was successful and you can start issuing SQL queries to the Employees database. For instance, try the below query. The results are shown in a table-like format.

```bash
mysql > SELECT * FROM employees LIMIT 5;
+--------+------------+------------+-----------+--------+------------+
| emp_no | birth_date | first_name | last_name | gender | hire_date  |
+--------+------------+------------+-----------+--------+------------+
|  10001 | 1953-09-02 | Georgi     | Facello   | M      | 1986-06-26 |
|  10002 | 1964-06-02 | Bezalel    | Simmel    | F      | 1985-11-21 |
|  10003 | 1959-12-03 | Parto      | Bamford   | M      | 1986-08-28 |
|  10004 | 1954-05-01 | Chirstian  | Koblick   | M      | 1986-12-01 |
|  10005 | 1955-01-21 | Kyoichi    | Maliniak  | M      | 1989-09-12 |
+--------+------------+------------+-----------+--------+------------+
5 rows in set (0.22 sec)
```

After the installation, we can use Python to connect to the employees database. Let's create a new connection.

In [ ]:
import mysql.connector as mysql

employees = mysql.connect(
    host = "localhost",
    user = "root",
    passwd = "vossensql",
    database = "employees"
)

We execute the SQL statement `SHOW TABLES` to list all tables in the employees database.

In [ ]:
import pandas as pd

pd.read_sql('SHOW TABLES', employees)

With the new connection we can start posing more complex queries to get insights from the data. For instance, let's find out how many female employees have been hired between 1980 and 1985.

In [ ]:
pd.read_sql(
    """SELECT COUNT(*) AS Women_80_85 FROM employees 
        WHERE gender = 'F' AND 
        hire_date between '1980-01-01' AND '1986-01-01'""", 
    employees
)

Since the result of `read_sql` is a data frame, we can start with a simple query and produce more complex analysis from Python directly, without the need to write complex SQL statements.

In [ ]:
hired_women = pd.read_sql(
    """SELECT * FROM employees 
        WHERE gender = 'F'""", 
    employees
)

hired_women.head()

In [ ]:
hired_women.info()

Let's convert date column to the appropriate datetime format. 

In [ ]:
hired_women.birth_date = pd.to_datetime(hired_women.birth_date, format='%Y-%m-%d')
hired_women.hire_date = pd.to_datetime(hired_women.hire_date, format='%Y-%m-%d')

Let's filter the results by `hire_date`.

In [ ]:
women_80_85 = hired_women[(hired_women.hire_date > '1980-01-01') & (hired_women.hire_date < '1986-01-01')]
women_80_85.head()

We get the count by calling `count()`.

In [ ]:
women_80_85.count()

Don't forget to close the connection to the database after performing your analysis. Otherwise the connection stays open.

In [ ]:
dbpy.close()
employees.close()

## Ende Note:

You can even use SQL to do data cleansing, see https://medium.com/@aakriti.sharma18/data-cleaning-with-sql-eaab6d29d007.

---

# End M2E03: Interfacing Python with SQLite and MySQL